**Instalando la libreria**

In [ ]:
!pip install tensorflowjs
!pip install tensorflow
!pip install imageio[ffmpeg]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.2/89.2 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 89.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 475.2/475.2 MB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 104.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 442.0/442.0 kB 49.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 96.1 MB/s eta 0:00:00
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.14.0
    Uninstalling tensorflow-estimator-2.14.0:
      Successfully uninstalled tensorflow-estimator-2.14.0
  Attempting uninstall: keras
    Found existing installation: keras 2.14.0
    Uninstalling keras-2.14.0:
      Successfully uninstalled keras-2.14.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.14.1
    Uninstalling tensorboard-2.14.1:
      Successfully uninstall

In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import imageio
import os
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import TensorBoard

**Función para cargar imágenes**

In [ ]:
def cargar_imagenes(ruta, tamano_comun):
    imagenes = []
    labels = []

    categorias = os.listdir(ruta)

    for idx, directorio in enumerate(categorias):
        for imagen in os.listdir(os.path.join(ruta, directorio)):
            img_path = os.path.join(ruta, directorio, imagen)
            try:
                img = Image.open(img_path).resize(tamano_comun)
            except UnidentifiedImageError:
                # Intentar abrir la imagen con imageio si Pillow no puede identificar el formato
                img = imageio.imread(img_path)
                img = Image.fromarray(np.uint8(img))

            # Convertir a formato RGB (descartando el canal alfa si existe)
            img = img.convert('RGB')

            img = np.asarray(img)

            # Asegurarse de que todas las imágenes tengan la misma forma (100, 100, 3)
            if img.shape == tamano_comun + (3,):
                imagenes.append(img)
                labels.append(idx)

    imagenes = np.asarray(imagenes)
    labels = np.asarray(labels)

    return imagenes, labels

**Función para normalizar imágenes**

In [ ]:
 def normalizar_imagenes(imagenes):
    return imagenes.astype(float) / 255

**Función para definir y compilar el modelo**

In [ ]:
def crear_modelo(input_shape, num_categorias):
    modelo = tf.keras.models.Sequential([
        tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=input_shape),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.MaxPooling2D(2, 2),
        tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.MaxPooling2D(2,2),
        tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.MaxPooling2D(2, 2),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(100, activation='relu'),
        tf.keras.layers.Dense(num_categorias, activation="softmax")
    ])

    modelo.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
                   loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                   metrics=['accuracy'])

    return modelo

**Rutas relativas**

In [ ]:
ruta_categorias = '/content/drive/MyDrive/ProyectoIncendios/Redes/ProyectoInc/Cate'

**Cargar imágenes**

In [ ]:
imagenes, labels = cargar_imagenes(ruta_categorias, (100, 100))

**Normalizar imágenes**

In [ ]:
imagenes = normalizar_imagenes(imagenes)

**Dividir datos en entrenamiento y validación**

In [ ]:
X_entrenamiento, X_validacion, y_entrenamiento, y_validacion = train_test_split(imagenes, labels, test_size=0.15, random_state=42)

**Aumento de datos**

In [ ]:
datagen = ImageDataGenerator(
    rotation_range=45,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=20,
    zoom_range=[0.8, 1.2],
    horizontal_flip=True,
    vertical_flip=True
)
datagen.fit(X_entrenamiento)

**Definir y compilar el modelo**

In [ ]:
modelo_AU = crear_modelo((100, 100, 3), len(os.listdir(ruta_categorias)))

**Configurar TensorBoard**

In [ ]:
 tensor_AU = TensorBoard(log_dir='logs/cnn_AD')

**Entrenar el modelo**

In [ ]:
historia = modelo_AU.fit(
    datagen.flow(X_entrenamiento, y_entrenamiento, batch_size=32),
    epochs=250,
    validation_data=(X_validacion, y_validacion),
    steps_per_epoch=int(np.ceil(len(X_entrenamiento) / float(32))),
    validation_steps=int(np.ceil(len(X_validacion) / float(32))),
    callbacks=[tensor_AU]
)

Epoch 1/250
64/64 [==============================] - 52s 819ms/step - loss: 0.4601 - accuracy: 0.8077 - val_loss: 1.8671 - val_accuracy: 0.6278
Epoch 2/250
64/64 [==============================] - 51s 790ms/step - loss: 0.4204 - accuracy: 0.8170 - val_loss: 2.6289 - val_accuracy: 0.6278
Epoch 3/250
64/64 [==============================] - 49s 752ms/step - loss: 0.3922 - accuracy: 0.8243 - val_loss: 0.9163 - val_accuracy: 0.6444
Epoch 4/250
64/64 [==============================] - 51s 794ms/step - loss: 0.3699 - accuracy: 0.8489 - val_loss: 0.5065 - val_accuracy: 0.7250
Epoch 5/250
64/64 [==============================] - 52s 809ms/step - loss: 0.3662 - accuracy: 0.8445 - val_loss: 1.0778 - val_accuracy: 0.7028
Epoch 6/250
64/64 [==============================] - 47s 735ms/step - loss: 0.3327 - accuracy: 0.8557 - val_loss: 0.5088 - val_accuracy: 0.7528
Epoch 7/250
64/64 [==============================] - 53s 822ms/step - loss: 0.3601 - accuracy: 0.8445 - val_loss: 0.4030 - val_accuracy:

# Guardar el modelo

In [ ]:
modelo_AU.save('/content/drive/MyDrive/ProyectoIncendios/Redes/ProyectoInc/Incendios-Normal-cnn-ad.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


# Convertir a formato TensorFlow.js

In [ ]:
!pip install tensorflowjs

#conectar carpeta con imagenes

In [ ]:
!mkdir carpeta_salida

In [ ]:
!tensorflowjs_converter --input_format keras /content/drive/MyDrive/ProyectoIncendios/Redes/ProyectoInc/Incendios-Normal-cnn-ad.h5 carpeta_salida

2023-11-29 05:38:29.037792: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-29 05:38:29.037850: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-29 05:38:29.039362: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-29 05:38:30.056909: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
